<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/CNNBasedSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentence to Salience

In [ ]:
import pandas as pd
t = pd.read_csv('/content/drive/MyDrive/BBC Dataset/LDA Summarization/LDASummariestrain.csv') 

In [ ]:
t

In [ ]:
rawData = {}
rawSummaries = {}
    
# running count variable -- keeps track of the total size
totalSentences = 0 

In [ ]:
def sep(text):
  import nltk
  nltk.download('punkt')
  import re
  text= text.replace("\t"," ")
  text=re.sub(r'\.+', '.',text)
  k=0
  c=0
  p=0
  a_list=[]
  while k<len(text):
    if c==0 and text[k]=='.':
      a_list.append(text[p:k+1])
      p=k+1
    elif c==0 and text[k]=='"':
      c=1
    elif c==1 and text[k]=='"' and text[k-1]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"' and text[k-2]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"':
      c=0
    k=k+1
  if p<len(text)-1:
    a_list.append(text[p:len(text)])
  return a_list
  

In [ ]:
def spl(text):
  t=text.split('\n',1)
  if len(t)!=2:
    return ""
  else :
    return t[1]

In [ ]:
j=0
while j<len(t):
  a=sep(t['News'][j])
  a[0]=spl(a[0])
  a[0]=spl(a[0])
  if a[0]=="":
    del a[0]
  i=0
  while i<len(a):
    a[i] = a[i].replace('\n',' ')
    a[i] = a[i].replace('\t',' ')
    a[i]= a[i].replace(r'\.+', ".")
    a[i]= a[i].replace(r'\ +', " ")
    i=i+1

  b=sep(t['Summary'][j])
  b[0]=spl(b[0])
  b[0]=spl(b[0])
  if b[0]=="":
    del b[0]
  i=0
  while i<len(b):
    b[i] = b[i].replace('\n',' ')
    b[i] = b[i].replace('\t',' ')
    b[i]= b[i].replace(r'\.+', ".")
    b[i]= b[i].replace(r'\ +', " ")
    i=i+1
  
  rawData[j]=a
  rawSummaries[j]=b
  j=j+1


In [ ]:
!pip install rouge/requirements.txt
!pip install rouge-score
import numpy as np

def calc_rouge_scores(pred_summaries, gold_summaries, 
                                 keys=['rouge1', 'rouge2'], use_stemmer=True):
    #Calculate rouge scores
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(keys, use_stemmer= use_stemmer)
    n = len(pred_summaries)
    scores = [scorer.score(pred_summaries[j], gold_summaries[j]) for 
              j in range(n)] 
              
    #create dict
    dict_scores={}                                                            
    for key in keys:
        dict_scores.update({key: {}})
        
    #populate dict    
    for key in keys:
        
        precision_list = [scores[j][key][0] for j in range(len(scores))]
        recall_list = [scores[j][key][1] for j in range(len(scores))]
        f1_list = [scores[j][key][2] for j in range(len(scores))]

        precision = np.mean(precision_list)
        recall = np.mean(recall_list)
        f1 = np.mean(f1_list)
        
        dict_results = {'recall': recall, 'precision': precision, 'f1': f1}
        
        dict_scores[key] = dict_results
        
    return dict_scores

In [ ]:
saliency={}
i=0
while i<len(rawData):
  saliency[i]=[]
  j=0
  while j<len(rawData[i]):
    s=calc_rouge_scores([rawData[i][j]],[t['Summary'][i]])
    saliency[i].append(0.5*s['rouge1']['f1']+s['rouge2']['f1']*0.5) #alpha=0.5
    j=j+1
  i=i+1

In [ ]:
tot=0;
i=0
while i<len(rawData):
  tot=tot+len(rawData[i])
  i=i+1

In [ ]:
import numpy as np
nx3output = np.zeros((tot, 3), dtype=object)

In [ ]:
i=0
c=0
while i<len(rawData):
  j=0
  while j<len(rawData[i]):
    nx3output[c,0]=i
    nx3output[c,1]=rawData[i][j]
    nx3output[c,2]=saliency[i][j]
    j=j+1
    c=c+1
  i=i+1


In [ ]:
import pickle
f = open("/content/drive/MyDrive/BBC Dataset/CNN Summarisation/sentencesToSaliency.pickle", "wb")
pickle.dump(nx3output, f)

In [ ]:
f.close()

In [ ]:
t.to_csv('/content/drive/MyDrive/BBC Dataset/CNN Summarisation/CNNSummariestrain')